In [34]:
import os
import numpy as np
import seqdata as sd
import xarray as xr

In [3]:
# Set wd
os.chdir("/cellar/users/aklie/data/datasets/deAlmeida_DrosophilaS2_UMI-STARR-seq")

In [12]:
# Set output dir
outdir_path = "training/2023_12_19/basepair_res"

# Make output dir if it doesn't exist
if not os.path.exists(outdir_path):
    os.makedirs(outdir_path)

In [26]:
sdata_orig = sd.open_zarr("training/2023_12_19/seqdatasets/deAlmeida22_training.zarr")
sdata_orig

<xarray.Dataset>
Dimensions:                                  (_sequence: 442866, length: 249,
                                              _ohe: 4, _length: 249)
Dimensions without coordinates: _sequence, length, _ohe, _length
Data variables: (12/18)
    Dev_log2_enrichment                      (_sequence) float64 dask.array<chunksize=(55359,), meta=np.ndarray>
    Dev_log2_enrichment_quantile_normalized  (_sequence) float64 dask.array<chunksize=(55359,), meta=np.ndarray>
    Dev_log2_enrichment_scaled               (_sequence) float64 dask.array<chunksize=(55359,), meta=np.ndarray>
    Hk_log2_enrichment                       (_sequence) float64 dask.array<chunksize=(55359,), meta=np.ndarray>
    Hk_log2_enrichment_quantile_normalized   (_sequence) float64 dask.array<chunksize=(55359,), meta=np.ndarray>
    Hk_log2_enrichment_scaled                (_sequence) float64 dask.array<chunksize=(55359,), meta=np.ndarray>
    ...                                       ...
    seq                                      (_sequence, _length) |S1 dask.array<chunksize=(10000, 249), meta=np.ndarray>
    set                                      (_sequence) <U8 dask.array<chunksize=(13840,), meta=np.ndarray>
    start                                    (_sequence) <U8 dask.array<chunksize=(13840,), meta=np.ndarray>
    strand                                   (_sequence) <U1 dask.array<chunksize=(110717,), meta=np.ndarray>
    train_val                                (_sequence) bool dask.array<chunksize=(221433,), meta=np.ndarray>
    type                                     (_sequence) <U9 dask.array<chunksize=(13840,), meta=np.ndarray>

In [19]:
training_bed = sdata_orig[["chrom", "start", "end"]].to_dataframe()

In [20]:
training_bed.to_csv(os.path.join(outdir_path, "training.bed"), sep="\t", index=False, header=False)

In [23]:
# Drosophila fasta file
fasta_file = "/cellar/users/aklie/data/ref/genomes/dm3/dm3.fa"
bed_file = os.path.join(outdir_path, "training.bed")

In [24]:
# Compose a SeqData from a set of files - only run this once!
sdata = sd.from_region_files(
    sd.GenomeFASTA(
        'seq',
        fasta_file,
        batch_size=2048,
        n_threads=4,
    ),
    sd.BigWig(
        'cov',
        ['analysis/2023_12_11/geo/GSE183936_S2_dev_STARRseq_merged.bw', 'analysis/2023_12_11/geo/GSE183936_S2_hk_STARRseq_merged.bw'],
        ['dev', 'hk'],
        batch_size=2048,
        n_jobs=4,
        threads_per_job=2,
    ),
    path=os.path.join(outdir_path, "deepstarr_training_bpres.zarr"),
    fixed_length=2114,
    bed=bed_file,
    overwrite=True,
    max_jitter=128
)

100%|██████████| 442866/442866 [00:17<00:00, 25368.80it/s]
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
100%|██████████| 442866/442866 [01:15<00:00, 5836.27it/s]


In [42]:
# Load SeqData
sdata = sd.open_zarr("training/2023_12_19/basepair_res/deepstarr_training_bpres.zarr")
sdata

<xarray.Dataset>
Dimensions:     (_sequence: 442866, cov_sample: 2, _length: 2370)
Coordinates:
  * cov_sample  (cov_sample) object 'dev' 'hk'
Dimensions without coordinates: _sequence, _length
Data variables:
    chrom       (_sequence) object dask.array<chunksize=(55359,), meta=np.ndarray>
    chromEnd    (_sequence) int64 dask.array<chunksize=(55359,), meta=np.ndarray>
    chromStart  (_sequence) int64 dask.array<chunksize=(55359,), meta=np.ndarray>
    cov         (_sequence, cov_sample, _length) uint16 dask.array<chunksize=(2048, 1, 2370), meta=np.ndarray>
    seq         (_sequence, _length) |S1 dask.array<chunksize=(2048, 2370), meta=np.ndarray>
    strand      (_sequence) object dask.array<chunksize=(55359,), meta=np.ndarray>
Attributes:
    length_dim:    _length
    max_jitter:    128
    sequence_dim:  _sequence

In [43]:
sdata["train_val"] = sdata_orig["train_val"]
sdata["train_val"].to_series().value_counts()

True     402296
False     40570
Name: train_val, dtype: int64

In [44]:
# Need to upper case the seqs for ohe
sdata["cleaned_seq"] = xr.DataArray(np.char.upper(sdata["seq"]), dims=["_sequence", "_length"])

In [49]:
# Check how many b'N' characters exist in "cleaned_seq"
(sdata["cleaned_seq"] == b"N").sum().values

array(203080)

In [50]:
sdata

<xarray.Dataset>
Dimensions:      (_sequence: 442866, cov_sample: 2, _length: 2370)
Coordinates:
  * cov_sample   (cov_sample) object 'dev' 'hk'
Dimensions without coordinates: _sequence, _length
Data variables:
    chrom        (_sequence) object dask.array<chunksize=(55359,), meta=np.ndarray>
    chromEnd     (_sequence) int64 dask.array<chunksize=(55359,), meta=np.ndarray>
    chromStart   (_sequence) int64 dask.array<chunksize=(55359,), meta=np.ndarray>
    cov          (_sequence, cov_sample, _length) uint16 dask.array<chunksize=(2048, 1, 2370), meta=np.ndarray>
    seq          (_sequence, _length) |S1 dask.array<chunksize=(2048, 2370), meta=np.ndarray>
    strand       (_sequence) object dask.array<chunksize=(55359,), meta=np.ndarray>
    train_val    (_sequence) bool dask.array<chunksize=(221433,), meta=np.ndarray>
    cleaned_seq  (_sequence, _length) |S1 b'C' b'T' b'C' b'T' ... b'A' b'G' b'C'
Attributes:
    length_dim:    _length
    max_jitter:    128
    sequence_dim:  _sequence

In [57]:
# Split cov into hk and dev
sdata['hk'] = (
    sdata.cov.sel(cov_sample=['hk'])
    .rename({'cov_sample': 'cov_split'})
    .assign_coords({'cov_split': ['hk']})
)
sdata['dev'] = (
    sdata.cov.sel(cov_sample=['dev'])
    .rename({'cov_sample': 'cov_split'})
    .assign_coords({'cov_split': ['dev']})
)

In [66]:
sdata["chrom"] = sdata["chrom"].astype(str)
sdata["strand"] = sdata["strand"].astype(str)
sdata["cov_sample"] = sdata["cov_sample"].astype(str)

In [67]:
sdata.to_zarr(os.path.join(outdir_path, "deepstarr_training_bpres.zarr"), mode="w")

# DONE!

---